Purpose
====

In this document we explore some of the differences between Python, Cython, and NumPy for optimized calcutions.

Various implementations of [Conway's game of life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) will be used as examples.

Please see the doctring of each _iterate_ function for more information.

Sources
--------

Nicolas P. Rougier: [Numpy tutorial](http://www.labri.fr/perso/nrougier/teaching/numpy/numpy.html)

From StackExchange and StackOverflow: [Fastest way to iterate over NumPy arrays](http://codereview.stackexchange.com/questions/38580/fastest-way-to-iterate-over-numpy-array), [Python List Comprehension Vs. Map](http://stackoverflow.com/questions/1247486/python-list-comprehension-vs-map), [Optimizing ... Game of Life [in] NumPy](http://codereview.stackexchange.com/questions/46011/optimizing-very-simple-piece-of-game-of-life-code-by-taking-advantage-of-numpy)

From reddit's /r/learnpython: [Cython optimization ...](https://www.reddit.com/r/learnpython/comments/3g73ux/cython_optimization_can_i_get_any_closer/)

In [1]:
%load_ext Cython
import os, sys, copy
from collections import namedtuple
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from numpy import log, exp, minimum
from matplotlib import pylab as plt
import cPickle as pickles
%alias_magic t timeit

Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.


In [2]:
def view_results(src, dst, c, m):
    '''Load Z_all from pickle. Save Z_all to disk as images.'''
    
    Z_all = pickle.load(open(src, 'rb'))
    
    fig = plt.figure(figsize=m.figsize, dpi=m.dpi, facecolor="white")
    ax1 = fig.add_axes([0.0, 0.0, 1.0, 1.0], frameon=False)

    for i in range(c.n_iterations):
        ax1.imshow(Z_all[i], interpolation='nearest', cmap=plt.cm.gray_r)
        
        fig.savefig('./{}/img_{}.png'.format(dst, i))
        ax1.cla()

    plt.close(fig)

In [3]:
def iterate_v1(Z, c):
    '''Naive element by element iteration.
    
    Args:
        Z (ndarray) - Represents 2D space
        c (namedtuple) - Container for constants
    Returns:
        Z (ndarray)
    '''
    
    # Count neighbors
    N = np.zeros((c.rows-1, c.cols-1), dtype=np.int32)
    
    for x in range(1, c.cols-1):
        for y in range(1, c.rows-1):
            N[y][x] = (Z[y-1][x-1]+Z[y][x-1]+Z[y+1][x-1] +
                       Z[y-1][x]            +Z[y+1][x]   +
                       Z[y-1][x+1]+Z[y][x+1]+Z[y+1][x+1])
    # Apply rules
    for x in range(1, c.cols-1):
        for y in range(1, c.rows-1):
            if Z[y][x] == 1 and (N[y][x] < 2 or N[y][x] > 3):
                Z[y][x] = 0
            elif Z[y][x] == 0 and N[y][x] == 3:
                Z[y][x] = 1
    return Z

In [4]:
def iterate_v2(Z, c):
    '''Element by elemenent iteration with NumPy indexing.
    
    Args:
        Z (ndarray) - Represents 2D space
        c (namedtuple) - Container for constants
    Returns:
        Z (ndarray)
    '''
    
    # Count neighbors
    N = np.zeros((c.rows-1, c.cols-1), dtype=np.int32)
    
    for x in range(1, c.rows-1):
        for y in range(1, c.cols-1):
            N[x, y] = (Z[x-1, y-1] + Z[x-1, y] + Z[x-1, y+1] +
                       Z[x,   y-1]             + Z[x,   y+1] +
                       Z[x+1, y-1] + Z[x+1, y] + Z[x+1, y+1])
    # Apply rules
    for x in range(1, c.rows-1):
        for y in range(1, c.cols-1):
            if Z[x, y] == 1 and (N[x, y] < 2 or N[x, y] > 3):
                Z[x, y] = 0
            elif Z[x, y] == 0 and N[x, y] == 3:
                Z[x, y] = 1
    return Z

In [5]:
def iterate_v3(Z, c):
    '''NumPy vectorization and boolean arrays.

    Args:
        Z (ndarray) - Represents 2D space
        c (namedtuple) - Container for constants
    Returns:
        Z (ndarray)
    '''

    # Count neighbours
    N = (Z[0:-2,0:-2] + Z[0:-2,1:-1] + Z[0:-2,2:] +
         Z[1:-1,0:-2]                + Z[1:-1,2:] +
         Z[2:  ,0:-2] + Z[2:  ,1:-1] + Z[2:  ,2:])

    # Apply rules
    birth = (N==3) & (Z[1:-1, 1:-1]==0)
    survive = ((N==2) | (N==3)) & (Z[1:-1, 1:-1]==1)
    Z[...] = 0
    Z[1:-1,1:-1][birth | survive] = 1

    return Z

In [6]:
%%cython
import numpy as np
cimport numpy as np

cpdef iterate_v4(Z, c):
    '''Element by elemenent iteration with NumPy indexing and untyped Cython.

    Args:
        Z (ndarray) - Represents 2D space
        c (namedtuple) - Container for constants
    Returns:
        Z (ndarray)
    '''

    # Count neighbors
    N = np.zeros((c.rows-1, c.cols-1), dtype=np.int32)

    for x in range(1, c.rows-1):
        for y in range(1, c.cols-1):
            N[x, y] = (Z[x-1, y-1] + Z[x-1, y] + Z[x-1, y+1] +
                       Z[x,   y-1]             + Z[x,   y+1] +
                       Z[x+1, y-1] + Z[x+1, y] + Z[x+1, y+1])
    # Apply rules
    for x in range(1, c.rows-1):
        for y in range(1, c.cols-1):
            if Z[x, y] == 1 and (N[x, y] < 2 or N[x, y] > 3):
                Z[x, y] = 0
            elif Z[x, y] == 0 and N[x, y] == 3:
                Z[x, y] = 1
    return Z

In [7]:
%%cython
#cython: wraparound=False, boundscheck=False, cdivision=True
#cython: profile=False, nonecheck=False, overflowcheck=False
#cython: cdivision_warnings=False, unraisable_tracebacks=False
import numpy as np
cimport numpy as np

cpdef iterate_v5(Z, c):
    '''Element by elemenent iteration with optimized Cython.

    Args:
        Z (ndarray - int32) - Represents 2D space
        c (namedtuple) - Container for constants
    Returns:
        Z (ndarray - int32)
    '''

    # Count neighbors
    N = np.zeros((c.rows-1, c.cols-1), dtype=np.int32)

    cdef int rows = c.rows
    cdef int cols = c.cols
    cdef int [:, :] N_ = N
    cdef int [:, :] Z_ = Z
    cdef int x, y

    with nogil:
        for x in range(1, rows-1):
            for y in range(1, cols-1):
                N_[x, y] = (Z_[x-1, y-1] + Z_[x-1, y] + Z_[x-1, y+1] +
                            Z_[x,   y-1]              + Z_[x,   y+1] +
                            Z_[x+1, y-1] + Z_[x+1, y] + Z_[x+1, y+1])
        # Apply rules
        for x in range(1, rows-1):
            for y in range(1, cols-1):
                if Z_[x, y] == 1 and (N_[x, y] < 2 or N_[x, y] > 3):
                    Z_[x, y] = 0
                elif Z_[x, y] == 0 and N_[x, y] == 3:
                     Z_[x, y] = 1

    return np.array(Z_)

In [8]:
def time_me(c, lbl, iterate):
    '''This method exists only to allow easy use of timeit.'''
    
    Z = np.random.randint(0, 2, (c.rows, c.cols)).astype(np.int32)
    Z[0, :] = 0
    Z[-1, :] = 0
    Z[:, 0] = 0
    Z[:, -1] = 0
    
    Z_all = np.empty((c.n_iterations, c.rows, c.cols), dtype=np.int32)
    
    for i in range(c.n_iterations):
        Z_all[i, ...] = iterate(Z, c)
    
    #pickle.dump(Z_all, open('./results_{}.p'.format(lbl), 'wb'))

In [9]:
def test_all_versions(versions, c):
    '''Iterate over Z, then save Z_all to disk if enabled.'''
    
    # global variables used here as a quick hack for timeit
    global lbl  
    global iterate 
    
    for lbl, iterate in versions.iteritems(): 
        %t -n1 time_me(c, lbl, iterate)

In [10]:
Const = namedtuple('c', ['rows', 'cols', 'nd', 'n_iterations'])
c = Const(rows=100, cols=100, nd=2, n_iterations=100)

dpi = 20.0
figsize = c.cols / float(dpi), c.rows / float(dpi)
Config_mpl = namedtuple('m', ['figsize', 'dpi'])
m = Config_mpl(figsize=figsize, dpi=dpi)

In [11]:
versions = {'v1': iterate_v1, 'v2': iterate_v2, 'v3': iterate_v3, 
            'v4': iterate_v4, 'v5': iterate_v5}

test_all_versions(versions, c)

1 loops, best of 3: 9.05 s per loop
1 loops, best of 3: 7.53 s per loop
1 loops, best of 3: 19.7 ms per loop
1 loops, best of 3: 7.17 s per loop
1 loops, best of 3: 7.75 ms per loop


In [13]:
# Make directories for images
if not os.path.exists('v1'):
    for i in range(1, 6):
        os.makedirs('v{}'.format(i))

for i in range(1, 6):
    view_results('results_v{}.p'.format(i), 'v{}'.format(i), c, m)